<a href="https://colab.research.google.com/github/wesley34/comp3414_course_material/blob/master/ch_9_basic_reinforcement_learning/DeepReinforcementLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deep Reinforcement Learning : in working-process , not complete version

You will learn the model tunning method in DRL

1.   Basic DQN
2.   N-Step DQN
3.   DDQN
4.   Nosiy DQN
5.   Priority Replay Buffer
6.   Dueling DQN
7.   Distribution perspective DQN 



## You may need


*   some belman equation and optimality
*   experience in using Q_Tabel and understand why it converge
*   python programming 
*   oop
*   wrapper
*   pytorch



### Atari Pong will be used as it is the most easiet game in OPENAI
https://gym.openai.com/envs/Pong-v0/

### Architecture 
### The best thing of this game is that, only simple cnn network can already make a very good result

### QUICK PREVIEW


1.   DQN   -  use 2 neural network to do prediction (impoprtant, at least know this)
2.   N-step DQN - use 2 nerual network to do prediction but different in belman equation: faster to converge
3.   DDQN - use 2  neural network to do preidction but differet in belman equation: doesn't overestimate -> more accurate
4.   Noisy - same as DQN, but self-driven in explore / exploit -> diverse in data in a elegent way -> better converge-speed
5.   Prioritized Replay Buffer - same as DQN, but it can select some good data (those data that the AI will lose frequently) : better in training -> converge faster
6.   Dueling DQN -> converge faster by seperation in network: seperate Q_Value to State Value + Action Value
7.   Distribution prespective in DQN -> use distribution to represent instead of using a scalar number to represent the goodness of certain state



### 1. Basic DQN

Important feature


---



*   Target Network and Normal Network
*   Loss function = (Q(s,a) - [r + decay * max TargetQ(s_next,a_next)]) ^2
*   Q(s,a) is an action value generated from Normal Network, where the network want to be an approximator to calculate Q value
*   Target Q(s,a) is an action value generated from Target network, also an approximator, and it is usually a older version network compare to normal one (except the first time), as it will copy the weighting in Normal Network after every-n iteration
  * "r" is transitional reward where the agent moved from current state "s" by action "a" to next state "s_next"
  * r + decay * max Target Q(s_next,a_next) === current moving reward + decay * next state best Action value === approximate current Q value , please think about it !
*   so both are trying to be a approximator
*   We use 2 approximators to help improving model
  *   Loss = (Approximation_1 - Approximation_2)^2
*   As r will keep fit into the model, it is like Table learning method, until one day it can converge, kind of theory due to dynamic programing and math, no prove is here
*   Hence we can sure that Q will converge , so why we need 2 different network???
  *   Sadly, it will not converge in many times
  *   The tabel will not forget thing but deep learning model will forget, as the model are generalizing things, think about what linear regression are doing, l2-norm and the MLEs ...
  *   Hence we create such a method
  *   We only update Normal Network , after n-step iteration, Target networks' weighting will be synced.
  *   2 advantages:
      *   The network will learn more thoroughly in comparing to the first one as the old value keeps there
      *   It won't chase its tail
      *   It is a bit hard to understand, a good link: https://stackoverflow.com/questions/54237327/why-is-a-target-network-required

*   All in all, you may skip the above but hardcode your mind that there requires 2 networks, such that
  *   Q(s,a) is generated from normal network ; Target-Q(s_next,a_next) is generated from target network
  *   After n-iterations , the weighting in target network will copy the normal network, a.k.a sync as normal network
  *   Loss function = (Q(s,a) - r + max Target-Q(s_next,a_next)^2


---
At least, hardcode your mind with the last 3 sentences




### Replay buffer

---


*   list (state,action,reward,next state) a.k.a (s,a,r,n_s) will be pushed to replay buffer zome
*   we get those data from the walking agent, it will walk with folloing rule
  *    we hardcore some probability to it, such that it may walk randomly , so it can freely access the world , a.k.a exploration
  *    the remaining action will be just following the largest-Action based on  the Normal network, a.k.a exploitation
*  Probability is based on such a statement === walk_randomly if ("episilon" < python_generated_probability) else walk_with_DQN 
*  "Episilon" is hardcoded and it will be increased after the game start
  *   Therefore, more free exploration in the beginning of training phrase, then we want more DQN-rule (exploitation) 

*  we need agent to collect data and push to replay buffer -> then we sample the data randomly from this. why don't we just use the current infomation and put into network directly and train, instead of using the network-rule to get a lot of data and then randomly select them to put into model?
  *  Due to the probabilty of stochastic gradient descent
      *  IID is required, but you know the data is not iid if our data_size is small and the data has relationship if we instantly use the current data to fit, as current_state and next_state are already in relationship. Hence not independent.
      *  IID is required, but the distribution of the train data and goal model is different, there are 2 distribution in our training data, as we allow them to do exploration and exploitation

 ---
###All in all, you may skip and hardcode yourself that 

1.   we need to put (s,a,r,n_s) to replay buffer zone for further deep-learinig usage
2.   (s,a,r,n_s) is got by 2 methods through our agent, one is walk freely, another one is walk based on the max-Action-Value of our deep-Learning model
3.   replay buffer zone is needed due to Stochastic gradient descent as it is a kind of IID problem, or rather think is is a math problem if you do not know what is iid.








In [ ]:
# Basic DQN is the weakest, so we will not use it
# Lets change something in DQN and then code it
# By coding the enhanced-version , you will also learn how to code a basic DQN , no worry

## 2. N-step DQN

*   Q(s,a) = r + decay * max Q(n_s,n_a) // we are talking in table from but not DL form
*   Q(s,a) = r + decay * max Q(n_s,n_a) + decay^2 + max Q(n_n_s,n_n_a) // n means next , n_n 
*   what if we know that in next_state, it also move the same action "a" as current 
    * equation becomes === Q(s,a) = r1 + decay * r2 + decay^2 + max Q(n_n_s,n_n_a)
means next next
*   Consider
    1.   Q(s1,a) = r1 + decay * Q(s2,a) # third iteration
    2.   Q(s2,a) = r2 + decay * Q(s3,a) # second iteration, we get Q(s3,a) from first iteration, we get this
    3.   Q(s3,a) = r3   # first iteration, we have this <-- read from here

*   Also consider
    1.   Q(s1,a) = r1 + decay * r2 + decay^2 * Q(s3,a) # second iteration
    2.   Q(s2,a) = r2 + decay * r3 # first iteration, we get this , we have both r2 and r3.
    3.   Q(s3,a) = r3   # first iteration, we have this <-- read from here
 
*   So we assume the second movement as the first one can reduce the calcuation time !!
*   We can also set "n" step, which you think those "n" step are same action also.
*   The trade is easy to be seen, the more step you set, the more bias it is , as it betrayed the belman eqaution === Q(s,a) = r + decay * max Q(n_s,n_a), where it is not getting the max as you skip (for example) 100 max calculation, unless you are lucky, the continuos 100 action is the same and they are the max-action-value
*   So 2,3,4 is good as the bias will be small, we just betray a little bit to belman equation but get a faster converging time !!!


---


## In short, at least hardcode youself 
  1.   n-step DQN means ignore the max calculations in the following n-step
  2.   it can accelerate the converging speed if you do it correctly : optimal number
  3.   large number will make poor strategy, so the DQN will learn a bad strategy from large replay buffer zone






In [ ]:
## Now we cann add n-step dqn feature, but we can add more feature first , hence the model will be better
## We will not code it also, but again, we will code n-step with other feature as well

### 3. Double DQN

### 4. Noisy DQN


###Description


*   we do not want to hardcode a episilon
*   we do it by putting noise inside the network
*   This method is described as "exploring features" instead of "exploration only " as episilon method


---
###Methods

1.   Independent Gaussian noise : save mu and sigma inside each layer, you may think mu is mean , sigma is standard deviation
2.   Factorized Gaussian noise : use 2 random vector, the first one is the same size as the network input vector, the second one is the same size as the network output layer => calcuate outer sum later



---
###Data monitoring
1. check SNR graph, signal-to-noise ratio on network layer
2. the value increase means variance increase == need more exploration on feature







## 5. Prioritized Experience Replay

## 6. Duel Network

### 7. Distribution prespective in DQN

In [ ]:
## TODO!!1

### Example
Remarks: If you just mix all the things together, it may not be good. The rule of thumb is to think carefully , test and retry


---
Tips: If you know those CNN architecture like yolo (v_3 or above) , RCNN 
(masked or later version),capsule,eff-net(But it is very slow fps in our computer in duckiebot), you may also stick them with RNN architecture as a time-series contribution or traditional statistic model like extra-tree,xgBoost,regression,SVM,... 


---

Recall: You should use optimal batches, optimal activation function but lite, optimal loss function, optimal neuron size. You should also prune the tree and add l1,l2,custom_loss on them Also, apply dropout and difference network. If you know EM-routing, you may use it as well. Don't go with the value(Hyper-parameter) too extreme!!




---


1.   Basic DQN 
2.   DDQN+N_STEP
3.   Rainbow architecture (DDQN+N_STEP+Noisy+Distributional+Dueling with prioritized Replay buffer zone)



## Some reading


1.   https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html (Pytorch offical example)
2.   https://www.youtube.com/watch?v=yFBwyPuO2Vg&feature=youtu.be　（distributional prespective）
3.   https://arxiv.org/abs/1707.06887 (original paper from distributional prespective)
4.   https://medium.com/%E9%9B%9E%E9%9B%9E%E8%88%87%E5%85%94%E5%85%94%E7%9A%84%E5%B7%A5%E7%A8%8B%E4%B8%96%E7%95%8C/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92-ml-note-reinforcement-learning-%E5%BC%B7%E5%8C%96%E5%AD%B8%E7%BF%92-dqn-%E5%AF%A6%E4%BD%9Catari-game-7f9185f833b0 (good start from here with math, sorry it is in chinese :< )
5.   https://medium.com/@jonathan_hui/rl-dqn-deep-q-network-e207751f7ae4 (also a nice start from here with little math)
6.   https://medium.com/@shmuma/summary-noisy-networks-for-exploration-c8ba6e2759c7 (Noisy Network, go to his github to check more!!)


---


## Some extra reading


1.   https://qiita.com/keisuke-nakata/items/1f6d4387fc6f01a5bcac (evolve from c51, in japanese :> )
2.   https://zhuanlan.zhihu.com/p/60949506 (In Chinese)
3.   https://medium.com/analytics-vidhya/distributional-reinforcement-learning-part-2-iqn-and-fqf-567fbc7a04d7 (Recommended)



In [ ]:
## TODO

### END


1.   At least know and remember how to code DQN please
2.   If you can learn more rather than DQN, it will be good
3.   I know it is demanding to learn things rather than DQN in short of time





In [ ]:
## TODO :>